In [54]:
%config IPCompleter.greedy=True

In [55]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

In [56]:
url = 'http://www.espncricinfo.com/rankings/content/page/211270.html'
response = requests.get(url)

In [57]:
soup = BeautifulSoup(response.text, 'lxml')
#print(soup.getText())

In [58]:
target_urls = []
ratings = soup.find_all('div', class_='ciPhotoContainer')
for rating in ratings:
    frames = rating.findChildren('iframe')
    for frame in frames:
        links = frame.findChildren('a')
        for link in links:
            if '"' in link.get('href'):
                url = link.get('href').replace('"', '')
            else:
                url = link.get('href')
            target_urls.append(url)

In [60]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('cricket_rankings.xlsx', engine='xlsxwriter')
for target_url in target_urls:
    match_type_list = []
    gender_list = []
    category_list = []
    ranking_list = []
    player_list = []
    country_list = []
    target = target_url.split('feed/')[1]
    match_type = target.split('/')[0]
    category = target.split('/')[1]
    if 'women' in match_type:
        gender = 'Womens'
        match_type = match_type.replace('women', '')
    else:
        gender = 'Mens'
    
    response = requests.get(target_url)
    soup = BeautifulSoup(response.text, 'lxml')
    #print(soup.prettify())
    rankings = soup.find_all('td', class_='menuteam')
    players = soup.find_all('td', class_='menuteam-feed')
    countries = soup.find_all('td', class_='tdpadr')
    for country in countries:
        if not country.getText().isnumeric():
            country_list.append(country.getText())
    for ranking, player in zip(rankings, players):
        ranking_list.append(int(ranking.getText()))
        player_list.append(player.getText())
        match_type_list.append(match_type.capitalize())
        gender_list.append(gender.capitalize())
        category_list.append(category.capitalize())
    cricket_rankings = {
        'Ranking': ranking_list, 
        'Player': player_list, 
        'Country': country_list, 
        'Match Type': match_type_list, 
        'Gender': gender_list, 
        'Category': category_list
    }
    cricket_frame = pd.DataFrame(cricket_rankings)
    #cricket_frame.set_index(keys = ['Match Type', 'Gender'], inplace = True)
    
    sheet = gender + '-' + match_type + '-' + category
    
    print(cricket_frame)
    print("\n")
    
    # Convert the dataframe to an XlsxWriter Excel object.
    cricket_frame.to_excel(writer, sheet_name = sheet)

    # Close the Pandas Excel writer and output the Excel file.
writer.save()

   Ranking              Player Country Match Type Gender Category
0        1            V. Kohli     IND       Test   Mens  Batting
1        2     K.S. Williamson      NZ       Test   Mens  Batting
2        3        S.P.D. Smith     AUS       Test   Mens  Batting
3        4         C.A. Pujara     IND       Test   Mens  Batting
4        5           J.E. Root     ENG       Test   Mens  Batting
5        6         D.A. Warner     AUS       Test   Mens  Batting
6        7       H.M. Nicholls      NZ       Test   Mens  Batting
7        8            D. Elgar      SA       Test   Mens  Batting
8        9  F.D.M. Karunaratne      SL       Test   Mens  Batting
9       10           Azhar Ali     PAK       Test   Mens  Batting


   Ranking           Player Country Match Type Gender Category
0        1        K. Rabada      SA       Test   Mens  Bowling
1        2    J.M. Anderson     ENG       Test   Mens  Bowling
2        3     P.J. Cummins     AUS       Test   Mens  Bowling
3        4   V.D. Ph

   Ranking          Player Country Match Type  Gender Category
0        1      S.W. Bates      NZ        T20  Womens  Batting
1        2     S.R. Taylor      WI        T20  Womens  Batting
2        3         H. Kaur     IND        T20  Womens  Batting
3        4    M.M. Lanning     AUS        T20  Womens  Batting
4        5   D.J.S. Dottin      WI        T20  Womens  Batting
5        6  J.I. Rodriques     IND        T20  Womens  Batting
6        7     B.L. Mooney     AUS        T20  Womens  Batting
7        8      A.J. Healy     AUS        T20  Womens  Batting
8        9          M. Raj     IND        T20  Womens  Batting
9       10     S. Mandhana     IND        T20  Womens  Batting


   Ranking         Player Country Match Type  Gender Category
0        1      M. Schutt     AUS        T20  Womens  Bowling
1        2   Poonam Yadav     IND        T20  Womens  Bowling
2        3  L.M. Kasperek      NZ        T20  Womens  Bowling
3        4  S. Ecclestone     ENG        T20  Womens  Bow